In [73]:
import pandas as pd
import numpy as np

The first step is to load our data into separate dataframes. We are collecting and analyzing datasets that concern countries in the EU, and key data points that are associated with transportation, such as train ridership, car usage, economic productivity, and spending on transportation infrastructure. The datasets we are pulling, which are from Eurostat and the Internationa Transport Forum, are as follows:
- **GDP per capita in PPS:** Contains EU countries, along with their GDP per capita from 2010 to 2021.
- **Population** on 1 January by sex, country of birth and group of citizenship from 2010 to 2021.
- **Number of passenger cars per 1,000 inhabitants** in each country from 2010 to 2021.
- **Number of passengers transported by railways** in each country from 2010 to 2021.
- **Total road spending** including investment, maintenance spending and capital value of road, rail, inland waterways, maritime ports and airports, from years 2000 to 2020.
- **Inland transportation investment** including investment expenditure on railways infrastructure, roads, and waterways from 2000 to 2020. Both road spending and inland transportation investment come from the same data source.

In [74]:
# Load all datasets from csv files into dataframes
gdp = pd.read_csv("datasets/gdp.csv")
population = pd.read_csv("datasets/populations.csv")
passenger_cars = pd.read_csv("datasets/cars_per_1000_inhabitants.csv")
passenger_railways = pd.read_csv("datasets/passengers_transported.csv")
transportation_investment = pd.read_csv("datasets/transportation_spending.csv")

In [75]:
# Drop columns that are not needed from each dataframe
gdp.drop(columns=['DATAFLOW', 'LAST UPDATE', 'freq', 'na_item', 'ppp_cat', 'OBS_FLAG'], inplace=True)
population.drop(columns=['DATAFLOW', 'LAST UPDATE', 'freq', 'indic_de', 'OBS_FLAG'], inplace=True)
passenger_cars.drop(columns=['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'OBS_FLAG'], inplace=True)
passenger_railways.drop(columns=['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'OBS_FLAG'], inplace=True)
transportation_investment.drop(columns=['COUNTRY', 'VARIABLE',
                             'MEASURE', 'YEAR', 'Unit Code',
                            'Unit', 'PowerCode Code',
                            'PowerCode', 'Reference Period Code',
                            'Reference Period', 'Flag Codes',
                            'Flags'], inplace=True)

In [76]:
# Rename columns for consistency across all dataframes
gdp.rename(columns={"geo": "country", "TIME_PERIOD": "year", "OBS_VALUE": "gdp_per_capita"}, inplace=True)
population.rename(columns={"geo": "country", "TIME_PERIOD": "year", "OBS_VALUE": "population"}, inplace=True)
passenger_cars.rename(columns={"geo": "country", "TIME_PERIOD": "year", "OBS_VALUE": "passenger_cars_per_1000"}, inplace=True)
passenger_railways.rename(columns={"geo": "country", "TIME_PERIOD": "year", "OBS_VALUE": "passengers_transported"}, inplace=True)
transportation_investment.rename(columns={"Country": "country",
                                           "Year": "year",
                                            "Value": "value",
                                            "Variable": 'variable',
                                            'Measure': 'measure'},
                                            inplace=True)

In [77]:
# Only include the columns where Euro is the determined measure, and where variable = "Total road spending" OR "Total inland transport infrastructure investment"
transportation_investment = transportation_investment[transportation_investment.measure == 'Euro']
transportation_investment = transportation_investment[transportation_investment.variable.isin(['Total road spending', 'Total inland transport infrastructure investment'])]

In [78]:
# Value counts to ensure we filtered properly
transportation_investment['measure'].value_counts()

Euro    1748
Name: measure, dtype: int64

In [79]:
# Value counts to ensure we filtered properly
transportation_investment['variable'].value_counts()

Total inland transport infrastructure investment    938
Total road spending                                 810
Name: variable, dtype: int64

In [80]:
# Drop Euro column because it's useless now that everything is in Euros
transportation_investment.drop(columns=['measure'], inplace=True)

In [81]:
# Preview all dataframes to ensure appropriate names and values are included
passenger_cars.head()

,country,year,passenger_cars_per_1000
0,AL,2013,118
1,AL,2014,131
2,AL,2015,140
3,AL,2016,152
4,AL,2017,147


In [82]:
passenger_railways.head()

,country,year,passengers_transported
0,AT,2010,10263.0
1,AT,2011,10778.0
2,AT,2012,11211.0
3,AT,2013,11804.0
4,AT,2014,11981.0


In [83]:
gdp.head()

,country,year,gdp_per_capita
0,AL,2010,29
1,AL,2011,30
2,AL,2012,30
3,AL,2013,29
4,AL,2014,30


In [84]:
population.head()

,country,year,population
0,AD,2011,78115
1,AD,2012,78115
2,AD,2013,76246
3,AD,2016,71732
4,AD,2018,74794


In [85]:
# Here the transportation_investment df includes both inland transportation and road spending
transportation_investment.head()

,country,variable,year,value
84,Albania,Total inland transport infrastructure investment,2000,1.104212e+08
85,Albania,Total inland transport infrastructure investment,2001,1.090002e+08
86,Albania,Total inland transport infrastructure investment,2002,7.009591e+07
87,Albania,Total inland transport infrastructure investment,2003,6.152459e+07
88,Albania,Total inland transport infrastructure investment,2004,9.316940e+07


In [86]:
# Convert country codes into countries

#Dictionary with codes as keys and names as values
country_codes = {
    "BE": "Belgium",
    "BG": "Bulgaria",
    "CZ": "Czechia",
    "DK": "Denmark",
    "EE": "Estonia",
    "IE": "Ireland",
    "EL": "Greece",
    "ES": "Spain",
    "FR": "France",
    "FX": "France (metropolitan)",
    "HR": "Croatia",
    "IT": "Italy",
    "CY": "Cyprus",
    "LV": "Latvia",
    "LT": "Lithuania",
    "LU": "Luxembourg",
    "HU": "Hungary",
    "MT": "Malta",
    "NL": "Netherlands",
    "AT": "Austria",
    "PL": "Poland",
    "PT": "Portugal",
    "RO": "Romania",
    "SI": "Slovenia",
    "SK": "Slovakia",
    "FI": "Finland",
    "SE": "Sweden",
    "IS": "Iceland",
    "LI": "Liechtenstein",
    "NO": "Norway",
    "CH": "Switzerland",
    "UK": "United Kingdom",
    "ME": "Montenegro",
    "MK": "North Macedonia",
    "AL": "Albania",
    "RS": "Serbia",
    "TR": "Turkey",
    "AD": "Andorra",
    "BY": "Belarus",
    "BA": "Bosnia",
    "XK": "Kosovo",
    "MD": "Moldova",
    "MC": "Monaco",
    "RU": "Russia",
    "SM": "San Marino",
    "UA": "Ukraine",
    "AM": "Armenia",
    "AZ": "Azerbaijan",
    "GE": "Georgia"
}